## Final Project Submission

Please fill out:
* Student name: Dirk Van Curan
* Student pace: Full time
* Scheduled project review date/time:  August 23, 2019 9:30 am
* Instructor name: Cristian
* Blog post URL:


In [ ]:
# Your code here - remember to use markdown cells for comments as well!

In [ ]:
"""This file contains some in progress code on our Mod 1 project.
In this project, we are given real estate sales data from King
County in Seattle, and we are asked to make a predictor for price."""

In [ ]:
# import modules needed for data analysis and get them ready for use in the notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

In [ ]:
# import the given data and store it in a pandas DataFrame
raw_data = pd.read_csv("kc_house_data.csv")

## Initial exploration of what the data looks like

In [ ]:
raw_data.shape

In [ ]:
raw_data.head(20)

In [ ]:
raw_data.info()

## Decided to use only the zipcodes of each sale as a initial model

In [ ]:
# Convert the zipcodes to an array of One Hot values instead of a column of integers
# Zipcodes are not really useful as integers since they are actually categorical 
# values and being in a zipcode or not is more helpful for the model
ohe = OneHotEncoder(drop='first', categories='auto') 

In [ ]:
price_zip_trans = ohe.fit_transform(raw_data['zipcode'].values.reshape(-1,1))

In [ ]:
ohe.get_feature_names()

In [ ]:
# this takes the One Hot encoded array and makes a new sparse DataFrame
# the column names are the zipcodes and the values are Boolean 1s or 0s
# depending on if the sale happened in that zipcode
zip_sparce = pd.DataFrame(price_zip_trans.todense(), columns=ohe.get_feature_names())

In [ ]:
# adding the sale prices as a new column to the sparse array so that it can be
# used in a model
zip_sparce['price'] = raw_data['price']

In [ ]:
# initial examination of correlation between price and zipcode
zip_sparce.corr()['price'].sort_values(ascending=False)

In [ ]:
price_array = np.asarray(zip_sparce['price'])

price_array

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize':(24,3)})
sns.heatmap(zip_sparce.corr().loc[['price']],vmax=0.3);

## Begin doing linear regression

In [ ]:
# Choose our predictors for the model
X = zip_sparce.drop('price', axis=1)
y = zip_sparce['price']
X.info()

In [ ]:
# Initial rough model for exploration
predictors = sm.add_constant(X)
model = sm.OLS(y, predictors).fit()
model.summary()

## Try with improved modeling (not StatsModels)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(zip_sparce.drop('price', axis=1), 
                                                    zip_sparce['price'],
                                                   random_state=42)

In [ ]:
# Let's create a StandardScaler object to scale our data for us.
ss = StandardScaler()

# Now we'll apply it to our data by using the .fit() and transform() methods.
ss.fit(X_train)
X_train_sc = ss.transform(X_train)
X_test_sc = ss.transform(X_test)


In [ ]:
# Now we can fit a LinearRegression object to our training data!

lr = LinearRegression()
lr.fit(X_train_sc, y_train)

In [ ]:
lr.score(X_test_sc, y_test)


In [ ]:
# We can use the .coef_ attribute to recover the results
# of the regression.

lr.coef_

In [ ]:
# And .intercept_

lr.intercept_

In [ ]:
X_train.columns

Erin suggested avoiding groupby to make a bunch of different dataframes. Instead, consider making "hybrid" columns that play with the data you are given to give another metric for the model to use.

Remember to clean the data as you go!

Think about the assumptions of the model you're using and if you need to transform your data

In [ ]:
test = raw_data.groupby('zipcode')

In [ ]:
test.reset_index()

In [ ]:
for name, group in test:
    print(name)
    print(group)

In [ ]:
test.get_group(98178)

In [ ]:
raw_data.columns

In [ ]:
test.info

In [ ]:
raw_data['view'].sum()

In [ ]:
raw_data['waterfront'].sum()

In [ ]:
test2 = raw_data.drop()

In [ ]:
raw_data.describe()

In [ ]:
plt.hist(raw_data['bedrooms'],range=(1,6));

# What to do with the columns that have null values?
waterfront: A: there are only 146 records on the waterfront. I think we can drop this column
view: A: drop this as I don't think this helps. If we convert the nulls to zero, it doesn't seem to affect the value of the column much. Before cleaning, there are only 5,036 records that have view out of ~21,600
yr_renovated: A: I think we can convert year renovated to the yr_built value if it is null and then make a new column called age_since_upgrade

Erin: What if you looked at sqftliving/ rooms?

In [ ]:
view_nan_dropped = raw_data.dropna(axis='rows', subset=['view', 'price'])
view_nan_dropped

In [ ]:
only_view_and_price = view_nan_dropped.filter(['view', 'price'], axis=1)
only_view_and_price

In [ ]:
only_view_and_price.corr()['price'].sort_values(ascending=False)

This is a surprisingly high correlation. We were expecting no correlation

In [ ]:
plt.hist(only_view_and_price['view']);

In [ ]:
only_4_views = raw_data.loc[raw_data['view'] == 4]

In [ ]:
plt.hist(only_4_views['price']);

In [ ]:
only_4_views['price'].describe()

I think the view data should be dropped as using it would break one of the assumptions of linear regression. Linear regression assumes that all the independent variables follow a normal distribution, but View is so heavily skewed right(?) that it should not be considered a normal distribution. Almost 20k of the 21k records have 0 views.


Dirk: These are actually categorical 

In [ ]:
raw_data['waterfront'].describe()

In [ ]:
plt.hist(raw_data['waterfront']);

A similar logic should be followed for Waterfront. There are only 146 records with a waterfront with ~3000 nan values. Even if the NaN values were replaced with 1, this would result in a bimodel distribution, not a normal one.

Not enough values with a weak correlation

Since we are looking at prices, the unique IDs should not be relevant for our model. That colummn should also be dropped.

In [ ]:
wf_nan_dropped = raw_data.dropna(axis='rows', subset=['waterfront', 'price'])
only_wf_and_price = wf_nan_dropped.filter(['waterfront', 'price'], axis=1)
only_wf_and_price.corr()['price'].sort_values(ascending=False)

In [ ]:
wf_replace_na_with_0 = raw_data.fillna(0)
only_wf_no_nan_and_price = wf_replace_na_with_0.filter(['waterfront', 'price'], axis=1)
only_wf_no_nan_and_price.corr()['price'].sort_values(ascending=False)

In [ ]:
wf_replace_na_with_0['waterfront'].sum()

In [ ]:
yr_built_price = raw_data.filter(['yr_built', 'price'])
yr_built_price.corr()['price'].sort_values(ascending=False)

In [ ]:
import datetime

In [ ]:
current_time = datetime.datetime.now()
type(current_time.year)

In [ ]:
yr_built_price['age_int'] =  current_time.year - raw_data['yr_built']

In [ ]:
yr_built_price = yr_built_price.drop('age_datetime',axis=1)

In [ ]:
yr_built_price.info()

In [ ]:
yr_built_price.corr()['price'].sort_values(ascending=False)

The correlation between price and age is weak and thus we should drop age from the model

In [ ]:
yrreno_replace_na_with_0 = raw_data.dropna(0)
only_reno_no_nan_and_price = yrreno_replace_na_with_0.filter(['yr_renovated', 'price'], axis=1)
only_reno_no_nan_and_price.corr()['price'].sort_values(ascending=False)

This is also a weak correlation, so we should drop this

In [ ]:
raw_data.columns

In [ ]:
working_data = raw_data.drop(['id', 'waterfront', 'date', 'yr_built', 
                              'yr_renovated', 'sqft_living15', 'sqft_lot15'], axis=1)

In [ ]:
working_data.head()

In [ ]:
working_data.corr()['price'].sort_values(ascending=False)

In [ ]:
working_data = working_data.drop('sqft per room', axis=1)

In [ ]:
working_data['lat_by_long'] = working_data['lat'] / working_data['long']

What if we did a one hot encode on view? It would hopefully reduce the importance of view rated 0 but significantly improve the impact of a view rated as 4
    The one hot encode on view reduced the correlation between a 4 view and price. If we keep view, we should not do a OHE

Should we one hot encode condition?
    Condition's correlation is very weak and did not improve by doing a one hot encode. We think we should drop condition as a parameter for the model

Can we hybridize view and grade? Or view and condition?

Figure out normalize for categories
    could do sqft_living, sqft_lot, bedrooms, bathrooms, floors 
    
Is there a way to hybridize sqft_living and sqft_lot?
    Add the values?
    Division? (living/floors)/lot or lot/living?
    Could also do lot - living/floor?
    
Can we use latitude and longitude? Would we scale to the most expensive lat/long location?

Whenever we do more basic correlation exploration, we should say why we're not doing a more detailed analysis
    Simple correlation to determine whether or not we should do a more in depth analysis.
        
    If the correlation is already weak, it isn't likely to help us later
        


In [ ]:
view_price_data = working_data['view'].fillna(0)
view_price_data

In [ ]:
price_view_trans = ohe.fit_transform(view_price_data.values.reshape(-1,1))
view_sparce = pd.DataFrame(price_view_trans.todense(), columns=ohe.get_feature_names())
view_sparce['price'] = working_data['price']
view_sparce.corr()['price'].sort_values(ascending=False)

In [ ]:
cond_price_data = working_data['condition']
price_cond_trans = ohe.fit_transform(cond_price_data.values.reshape(-1,1))
cond_sparce = pd.DataFrame(price_cond_trans.todense(), columns=ohe.get_feature_names())
cond_sparce['price'] = working_data['price']
cond_sparce.corr()['price'].sort_values(ascending=False)

In [ ]:
plt.hist(working_data['grade'], range=(0,4));

In [ ]:
grade_price_data = working_data['grade']
price_grade_trans = ohe.fit_transform(grade_price_data.values.reshape(-1,1))
grade_sparce = pd.DataFrame(price_grade_trans.todense(), columns=ohe.get_feature_names())
grade_sparce['price'] = working_data['price']
grade_sparce.corr()['price'].sort_values(ascending=False)

In [ ]:
mean_grade = working_data['grade'].mean()
working_data['grade'] - mean_grade
median_grade = working_data['grade'].median()
median_grade
(working_data['grade'] - median_grade)/median_grade
std_grade = working_data['grade'].std()

In [ ]:
view_grade_hybridize = working_data['view'].fillna(0) * (working_data['grade'] - median_grade)/std_grade
#view_grade_hybridize['price'] = working_data['price']

In [ ]:
type(working_data['price'])

In [ ]:
view_grade_dict = {'view_grade_hybrid': view_grade_hybridize, 'price': working_data['price']}
view_grade_df = pd.DataFrame(view_grade_dict)
view_grade_df.corr()['price'].sort_values(ascending=False)

Drop 33 bedroom houses


In [ ]:
plt.hist(np.log(working_data['price']));

In [ ]:
lat_long = working_data.filter(['lat', 'long'])
lat_long.head()

In [ ]:
working_data = working_data.drop(['view', 'lat', 'long', 'condition'], axis=1)

In [ ]:
minus_33_rooms = working_data[working_data['bedrooms'] != 33] 

In [ ]:
minus_33_rooms.describe()

In [ ]:
finder = minus_33_rooms[(working_data['bedrooms'] >= 8)]
finder

In [ ]:
raw_data['floors'].describe()

In [ ]:
plt.hist(raw_data['floors']);

In [ ]:
working_data = working_data.drop(['sqft_above'], axis=1)

In [ ]:
working_data.columns

In [ ]:
working_data = working_data.drop('lat_by_long', axis=1)

In [ ]:
working_data.columns

In [ ]:
minus_33_rooms = working_data[working_data['bedrooms'] != 33] 

In [ ]:
minus_33_rooms.describe()

# Have cleaned data, switching drivers